<a href="https://colab.research.google.com/github/Agentoma/MMAI550_neuralnetwork_A1/blob/main/Copy_of_1_StyleGAN_FaceEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding a latent StyleGAN encoding of a face

Modified from the Colab notebook assoicated with the video [Editing Faces using Artificial Intelligence](https://www.youtube.com/watch?v=dCKbRCUyop8) and the GitHub repository [StyleGAN Encoder - Pytorch Implementation](https://github.com/jacobhallberg/pytorch_stylegan_encoder).

By Hjalmar K Turesson


# Part I: Encoding images into StyleGAN's latent space

![alt text](https://miro.medium.com/max/1280/0*eeFaGLx96mlbQcrK.gif)

## Before you move on, make sure you have GPU acceleration enabled:
> ### Click 'Runtime' in the menu tab at the top
> ### Click 'Change runtime type'
> ### Make sure the hardware accelerator is set to 'GPU'

## GitHub repositories

We will use three GitHub repos to find the latent encoding of faces.
 * [StyleGAN Encoder - Pytorch Implementation](https://github.com/jacobhallberg/pytorch_stylegan_encoder.git): PyTorch version
 * [InterFace GAN](https://github.com/genforce/interfacegan.git)
 * [StyleGAN encoder](https://github.com/Puzer/stylegan-encoder): Older Tensorflow version from where we are going to use `align_images.py`

 ### Get StyleGAN Encoder - Pytorch Implementation

In [ ]:
!git clone https://github.com/jacobhallberg/pytorch_stylegan_encoder.git

Cloning into 'pytorch_stylegan_encoder'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 361 (delta 0), reused 0 (delta 0), pack-reused 358 (from 1)
Receiving objects: 100% (361/361), 56.89 MiB | 19.22 MiB/s, done.
Resolving deltas: 100% (185/185), done.


#### `cd` into the repo folder:

In [ ]:
cd /content/pytorch_stylegan_encoder

### Get InterFace GAN
It should be located inside the newly created folder `pytorch_stylegan_encoder`.

In [ ]:
!rm -rf InterFaceGAN  # remove the empty directory created by the previous step
!git clone https://github.com/genforce/interfacegan.git # clone the repo

The new directory is named `interfacegan` but `encode_image.py` (to be used later) from `pytorch_stylegan_encoder` requires it to be called `InterFaceGAN`. Thus, we have to rename it.

In [ ]:
import os
os.rename('interfacegan', 'InterFaceGAN') # os.rename(old_name, new_name)

### Get StyleGAN-encoder
We need this to align the captured images. We will be using the script `align_images.py` which will preprocess images by extracting and aligning faces.

In [ ]:
!git clone https://github.com/Puzer/stylegan-encoder.git

### Let's see the files inside the repo we just cloned:

In [ ]:
ls

## Folders for our images
We need to create folders for our captured and aligned images.

In [ ]:
mkdir aligned_images raw_images

# I. Get Images:

## Some tips for the images:


*   Use HD images (preferably > 1000x1000 pixels)
*   Make sure your face is not too small
*   Neutral expressions & front facing faces will give better results
*   Clear, uniform lighting conditions are also recommened



## Option 1: Upload Images manually (usually gives the best results)



*   Click the '>' icon in the panel on the top left
*   Go to the 'Files' tab
*   Unfold the pytorch_stylegan_encoder folder (left-click)
*   Right click the 'pytorch_stylegan_encoder/raw_images' folder and click "upload"
*   I'd recommend starting with 3 - 6 different images containing faces



## Option 2: Take images using your webcam

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
from PIL import Image
from datetime import datetime

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""

def take_photo(quality=1.0, size=(800,600)):
  display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  f = io.BytesIO(binary)
  img = np.asarray(Image.open(f))

  timestampStr = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
  filename = 'raw_images/photo_%s.jpeg' %timestampStr
  Image.fromarray(img).save(filename)
  print('Image captured and saved to %s' %filename)

In [ ]:
img = take_photo() # click the image to capture a frame!

## Let's check the contents of our image folder before we start:
#### (You can always manually delete images by right clicking on them in the file tab)

In [ ]:
from PIL import Image
import os
imgs = sorted(os.listdir('raw_images'))

print("Found %d images in %s" %(len(imgs), 'raw_images'))
if len(imgs) == 0:
  print("Upload images to the \"raw_images\" folder!")
else:
  print(imgs)

for img_path in imgs:
  img = Image.open('raw_images/' + img_path)

  w,h = img.size
  rescale_ratio = 256 / min(w,h)
  img = img.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
  display(img)

## Make sure we're using the right TensorFlow version (1.15):

# II. Auto-Align faces:
### This script wil:


1.   Look for faces in the images
2.   Crop out the faces from the images
3.   Align the faces (center the nose and make the eyes horizontal)
4.   Rescale the resulting images and save them in "aligned_images" folder

`align_images.py` is located in the folder `styleganencoder` so we have to give the relative path (styleganencoder/align_images.py) to the file when running it (or `cd` into that folder).



In [ ]:
!python stylegan-encoder/align_images.py raw_images/ aligned_images/ --output_size=1024

## Let's take a look at our aligned images:

In [ ]:
def display_folder_content(folder, res = 256):
  if folder[-1] != '/': folder += '/'
  for i, img_path in enumerate(sorted(os.listdir(folder))):
    if '.png' in img_path:
      display(Image.open(folder+img_path).resize((res,res)), 'img %d: %s' %(i, img_path))
      print('\n')

display_folder_content('aligned_images')

# Important, before moving on:
### Manually clean the `aligned_images` directory

> ### 1. Manually remove all 'bad' images that are not faces / don't look sharp / clear
> #####  (Use the image names from the plots above to guide you)
> ### 2. Make sure you don't have too many faces in this folder (8 at most preferably)




# Encoding faces into StyleGAN latent space:

![title](https://raw.githubusercontent.com/pbaylies/stylegan-encoder/master/mona_example.jpg)

## Download a pretrained and fine-tuned the ResNet encoder.
This model takes an image as input and estimates the corresponding latent code.

In [ ]:
!gdown https://github.com/jacobhallberg/pytorch_stylegan_encoder/releases/download/v1.0/trained_models.zip

The model is called `image_to_latent.py` and is downloaded together with the StyleGAN model (`stylegan_ffhq.pth`) in the archive `trained_models.zip`.

We'll extract the models and move them to folders where `encode_images.py` can find them.

In [ ]:
!unzip trained_models.zip
!mv trained_models/stylegan_ffhq.pth InterFaceGAN/models/pretrain/
!mv trained_models/image_to_latent.pt .

# III. The actual encoding: `encode_images.py`

#### Note: This script will also download:

*   A pretrained VGG-16 network, trained on ImageNet

#### After guessing the initial latent codes using the pretrained ResNet, it will run gradient descent to optimize the latent faces!

In [ ]:
print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))
print("Recommended batch_size for the encode_images process: %d" %min(len(os.listdir('aligned_images/')), 8))

### Depending on the settings, the encoding process might take a few minutes...

In [ ]:
!python encode_image.py aligned_images/aligned1.png dlatents.npy --save_optimized_image true --use_latent_finder true --image_to_latent_path ./image_to_latent.pt

## Generate images from the latent encodings

Let's load the StyleGAN network into memory:

In [ ]:
from InterFaceGAN.models.stylegan_generator import StyleGANGenerator
from models.latent_optimizer import PostSynthesisProcessing

In [ ]:
synthesizer = StyleGANGenerator("stylegan_ffhq").model.synthesis
post_processing = PostSynthesisProcessing()

In [ ]:
import numpy as np
import torch
latent = torch.tensor(np.load('dlatents.npy'))
latent = latent.to(device='cuda')

In [ ]:
# Generate/synthezie from latent encoding
pred_image = synthesizer(latent)
# post process for better image quality
pred_image = post_processing(pred_image).detach().cpu().numpy().astype(np.uint8)

In [ ]:
# pred_image = pred_image.detach().cpu().numpy().astype(np.uint8)

In [ ]:
import matplotlib.pyplot as plt

original_image = plt.imread('aligned_images/aligned1.png')

fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(2,1,1)
ax.imshow(original_image)
ax.set_title('original image')
ax.set_xticks([])
ax.set_yticks([])
ax = fig.add_subplot(2,1,2)
pred_image = pred_image.squeeze()
ax.imshow(np.transpose(pred_image, (1,2,0)))
ax.set_title('predicted image')
ax.set_xticks([])
ax.set_yticks([])

## How can we generate random images?

# IV. Cherry pick images & dump their latent vectors to disk
### Manipulating latent vectors (Notebook II) is tricky and will only work well if the face encoding looks 'good'
### Cherry pick a few images where the optimization worked well
> (Use the image indices from the plot titles above)

## Save these latent vectors to disk:

# V. Manipulating the faces
### Everything we downloaded / saved to disk is currently on a temporary VM running on Google Colab
> We'll want to reuse the latent vectors later, so you should download them manually:
>> * Go to the root directory using the Files browser
>> * Richt-click & Download the latent representations: "output_vectors.npy"
## Next, let's continue with notebook II:
> ### Simply open the second notebook from the Drive folder and continue the guide-steps
> ### (Hint: Notebook II is where all the fun is!)

![alt text](https://66.media.tumblr.com/tumblr_mc3hg5VpQP1qcy0p7o1_400.gif)